# Data cleaning


In [36]:
# importing needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [37]:
#Loading the dataset
df = pd.read_csv('../data/new_retail_data.csv',parse_dates=["Date"])

In [38]:
df.shape

(302010, 30)

In [39]:
df.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302010 entries, 0 to 302009
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Transaction_ID    301677 non-null  float64       
 1   Customer_ID       301702 non-null  float64       
 2   Name              301628 non-null  object        
 3   Email             301663 non-null  object        
 4   Phone             301648 non-null  float64       
 5   Address           301695 non-null  object        
 6   City              301762 non-null  object        
 7   State             301729 non-null  object        
 8   Zipcode           301670 non-null  float64       
 9   Country           301739 non-null  object        
 10  Age               301837 non-null  float64       
 11  Gender            301693 non-null  object        
 12  Income            301720 non-null  object        
 13  Customer_Segment  301795 non-null  object        
 14  Date

In [43]:
df_clean = df.drop_duplicates(keep='first')
df = df_clean

In [7]:
cat_cols = df.select_dtypes(include="object").columns

for col in cat_cols:
    print(f"\n--- {col.upper()} ---")
    print(df[col].value_counts(dropna=False).head(30))



--- NAME ---
Name
NaN                    382
Michael Smith          144
Michael Johnson        111
Michael Jones          102
Christopher Smith       93
Michael Williams        91
Jennifer Smith          91
James Smith             89
Robert Smith            88
John Smith              84
David Smith             81
James Johnson           79
Joseph Smith            77
Michael Brown           76
Jennifer Johnson        75
David Williams          74
John Johnson            73
Jessica Smith           69
Jennifer Williams       68
David Johnson           68
William Smith           68
Christopher Johnson     66
John Williams           65
Michael Davis           64
Jennifer Brown          61
Michael Miller          60
James Jones             59
James Williams          59
Amanda Smith            58
Jessica Johnson         58
Name: count, dtype: int64

--- EMAIL ---
Email
NaN                    347
Michael59@gmail.com     93
Michael95@gmail.com     92
Michael17@gmail.com     90
Michael39@gmail.

In [8]:
#These are safe to drop
drop_cols = [
    "Transaction_ID",
    "Customer_ID",
    "Name",
    "Email",
    "Phone",
    "Address",
    "City",
    "Zipcode",
    "Age",
    "Gender",
    "Income",
    "Customer_Segment",
    "Year",          # not needed we will extract this later from Date
    "Month",         # not needed we will extract this later from Date
    "Time",
    "Amount",
    "Total_Amount",
    "Feedback",
    "Shipping_Method",
    "Payment_Method",
    "Order_Status",
    "Ratings"
]


df = df.drop(columns=[c for c in drop_cols if c in df.columns])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302010 entries, 0 to 302009
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   State             301729 non-null  object        
 1   Country           301739 non-null  object        
 2   Date              301651 non-null  datetime64[ns]
 3   Total_Purchases   301649 non-null  float64       
 4   Product_Category  301727 non-null  object        
 5   Product_Brand     301729 non-null  object        
 6   Product_Type      302010 non-null  object        
 7   products          302010 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 18.4+ MB


In [9]:
df.isnull().sum()

State               281
Country             271
Date                359
Total_Purchases     361
Product_Category    283
Product_Brand       281
Product_Type          0
products              0
dtype: int64

In [10]:
cols_required = [
    'Date',
    'products',
    'Total_Purchases',
    'Product_Category',
    'Product_Brand',
    'Product_Type',
    'Country',
    'State'
]

df_clean = df.dropna(subset=cols_required).reset_index(drop=True)
df=df_clean

In [11]:
# Fill missing categorical values
cat_cols = ["State","Country",
            "Product_Category","Product_Brand","Product_Type","products"]

for col in cat_cols:
    df[col] = df[col].astype(str).str.lower().str.strip()

In [12]:
# Fix spelling errors in Product Brand
df["Product_Brand"] = df["Product_Brand"].replace({
    "whirepool": "whirlpool",
    "mitsubhisi": "mitsubishi",
    "bluestar": "blue star"
})

In [13]:
#"uk" → "united kingdom"

df["Country"] = df["Country"].replace({
    "uk": "united kingdom"
})

In [14]:
# Convert "home decor" → "home_decor"
# Spaces in category names can cause issues in ML encoding.

df["Product_Category"] = df["Product_Category"].str.replace(" ", "_")

In [15]:
# Convert "blue star" → "blue_star"

df["Product_Brand"] = df["Product_Brand"].str.replace(" ", "_")

In [16]:
# Replace spaces with underscores:

df["Product_Type"] = df["Product_Type"].str.replace(" ", "_")

In [17]:
# Because models do better with clean tokens:

df["products"] = (
    df["products"]
    .str.lower()
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("[^a-zA-Z0-9_]", "", regex=True)
)

In [18]:
# Unique values + counts for "products"
print("\n🔹 Unique Products and Their Counts:")
product_counts = df['products'].value_counts()
print(product_counts)
print("\nTotal unique products:", product_counts.shape[0])

# Unique values + counts for "Product_Type"
print("\n🔹 Unique Product Types and Their Counts:")
ptype_counts = df['Product_Type'].value_counts()
print(ptype_counts)
print("\nTotal unique product types:", ptype_counts.shape[0])


🔹 Unique Products and Their Counts:
products
spring_water       2499
bottled_water      2490
mystery            2484
distilled_water    2463
artesian_water     2463
                   ... 
screwdriver_set     556
towel_rack          555
razer_blade         552
parka               545
package_ac          223
Name: count, Length: 318, dtype: int64

Total unique products: 318

🔹 Unique Product Types and Their Counts:
Product_Type
water                                 24352
smartphone                            18358
non-fiction                           18065
fiction                               17954
juice                                 12157
television                            12120
decorations                           12104
t-shirt                               12103
shoes                                 12067
tablet                                12014
soft_drink                            11960
furniture                             11901
fridge                                 7

In [19]:
df.head(10)

,State,Country,Date,Total_Purchases,Product_Category,Product_Brand,Product_Type,products
0,berlin,germany,2023-09-18,3.0,clothing,nike,shorts,cycling_shorts
1,england,united kingdom,2023-12-31,2.0,electronics,samsung,tablet,lenovo_tab
2,new south wales,australia,2023-04-26,3.0,books,penguin_books,children's,sports_equipment
3,ontario,canada,2023-05-08,7.0,home_decor,home_depot,tools,utility_knife
4,england,united kingdom,2024-01-10,2.0,grocery,nestle,chocolate,chocolate_cookies
5,new south wales,australia,2023-09-21,4.0,electronics,apple,tablet,lenovo_tab
6,ontario,canada,2023-06-26,2.0,electronics,samsung,television,qled_tv
7,berlin,germany,2023-03-24,1.0,clothing,zara,shirt,dress_shirt
8,new south wales,australia,2024-01-06,8.0,grocery,nestle,chocolate,dark_chocolate
9,berlin,germany,2023-10-04,10.0,home_decor,home_depot,decorations,candles


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300179 entries, 0 to 300178
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   State             300179 non-null  object        
 1   Country           300179 non-null  object        
 2   Date              300179 non-null  datetime64[ns]
 3   Total_Purchases   300179 non-null  float64       
 4   Product_Category  300179 non-null  object        
 5   Product_Brand     300179 non-null  object        
 6   Product_Type      300179 non-null  object        
 7   products          300179 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 18.3+ MB


In [21]:
df.describe()

,Date,Total_Purchases
count,300179,300179.000000
mean,2023-08-30 16:23:04.933656320,5.359935
min,2023-03-01 00:00:00,1.000000
25%,2023-05-31 00:00:00,3.000000
50%,2023-08-31 00:00:00,5.000000
75%,2023-11-30 00:00:00,8.000000
max,2024-02-29 00:00:00,10.000000
std,NaN,2.868481


In [22]:
# converting from product name to unique product ID
df['product_id'] = df['products'].astype('category').cat.codes


In [23]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)
df['IsMonthEnd'] = df['Date'].dt.is_month_end.astype(int)


In [24]:
df.sample(5)

,State,Country,Date,Total_Purchases,Product_Category,Product_Brand,Product_Type,products,product_id,Year,Month,DayOfWeek,IsMonthStart,IsMonthEnd
34081,new south wales,australia,2023-10-02,1.0,books,penguin_books,non-fiction,memoir,178,2023,10,0,0,0
153500,georgia,usa,2023-06-23,2.0,electronics,sony,smartphone,sony_xperia,266,2023,6,4,0,0
227877,south dakota,usa,2023-08-03,6.0,books,random_house,literature,short_stories,253,2023,8,3,0,0
242067,england,united kingdom,2023-05-21,1.0,home_decor,ikea,furniture,bookshelf,36,2023,5,6,0,0
224038,berlin,germany,2023-04-05,5.0,books,penguin_books,non-fiction,memoir,178,2023,4,2,0,0


In [25]:
df.isnull().sum()

State               0
Country             0
Date                0
Total_Purchases     0
Product_Category    0
Product_Brand       0
Product_Type        0
products            0
product_id          0
Year                0
Month               0
DayOfWeek           0
IsMonthStart        0
IsMonthEnd          0
dtype: int64

In [26]:
df.shape

(300179, 14)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300179 entries, 0 to 300178
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   State             300179 non-null  object        
 1   Country           300179 non-null  object        
 2   Date              300179 non-null  datetime64[ns]
 3   Total_Purchases   300179 non-null  float64       
 4   Product_Category  300179 non-null  object        
 5   Product_Brand     300179 non-null  object        
 6   Product_Type      300179 non-null  object        
 7   products          300179 non-null  object        
 8   product_id        300179 non-null  int16         
 9   Year              300179 non-null  int32         
 10  Month             300179 non-null  int32         
 11  DayOfWeek         300179 non-null  int32         
 12  IsMonthStart      300179 non-null  int64         
 13  IsMonthEnd        300179 non-null  int64         
dtypes: d

In [28]:
df.to_csv('../data/cleaned_retail_data.csv', index=False)

In [29]:
df.head()

,State,Country,Date,Total_Purchases,Product_Category,Product_Brand,Product_Type,products,product_id,Year,Month,DayOfWeek,IsMonthStart,IsMonthEnd
0,berlin,germany,2023-09-18,3.0,clothing,nike,shorts,cycling_shorts,86,2023,9,0,0,0
1,england,united kingdom,2023-12-31,2.0,electronics,samsung,tablet,lenovo_tab,164,2023,12,6,0,1
2,new south wales,australia,2023-04-26,3.0,books,penguin_books,children's,sports_equipment,270,2023,4,2,0,0
3,ontario,canada,2023-05-08,7.0,home_decor,home_depot,tools,utility_knife,303,2023,5,0,0,0
4,england,united kingdom,2024-01-10,2.0,grocery,nestle,chocolate,chocolate_cookies,58,2024,1,2,0,0
